In [1]:
import pandas as pd
import numpy as np

In [2]:
from ipynb.fs.full.extract_raw_data import extra_raw_data
from ipynb.fs.full.extract_raw_data import generate_balanced_aa_antigen_table

ipynb.fs.full.extract_raw_data:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipynb.fs.full.extract_raw_data:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipynb.fs.full.extract_raw_data:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipynb.fs.full.extract_raw_data:

In [3]:
# load data
dt = extra_raw_data()
balanced_aa_antigen_table = generate_balanced_aa_antigen_table(dt)

In [4]:
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=6):
    return [sequence[x:x+size].upper() for x in range(len(sequence) - size + 1)]

In [5]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
# plot ROC curve
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [6]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1

In [7]:
 # Convert our data sequences into short overlapping k-mers of legth 6 using our getKmers function.
balanced_aa_antigen_table['aa_words'] =  balanced_aa_antigen_table.apply(lambda x: getKmers(x['aa_1TRB_only'], size = 6), axis=1)


aa_texts = list(balanced_aa_antigen_table['aa_words'])
for item in range(len(balanced_aa_antigen_table)):
    aa_texts[item] = ' '.join(aa_texts[item])
    


In [8]:
# Creating the Bag of Words model using CountVectorizer()
# The n-gram size of 5 was previously determined by testing
ngram = 5

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(ngram,ngram))
X = cv.fit_transform(aa_texts)

In [9]:
y = balanced_aa_antigen_table['classes']

In [10]:
from sklearn.model_selection import train_test_split 
    
# Splitting the dataset into the training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state=42) 

In [11]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously

classifier = MultinomialNB(alpha=0.01)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [12]:
word_list = cv.get_feature_names();    
count_list = X.toarray().sum(axis=0) 
freq = dict(zip(word_list,count_list))
freq_table = pd.DataFrame.from_dict(freq, orient='index', columns=['frequency'])
freq_table = freq_table.sort_values(by='frequency', ascending=False)

In [13]:
print('total number of features = ' + str(len(word_list)))

total number of features = 141434


In [14]:
print('top 10 frequent features:')
freq_table.head()

top 10 frequent features:


,frequency
dleany leanyg eanygy anygyt nygytf,2801
svdlea vdlean dleany leanyg eanygy,2801
csvdle svdlea vdlean dleany leanyg,2801
vdlean dleany leanyg eanygy anygyt,2801
sfsgnt fsgntg sgntge gntgel ntgelf,1609


In [15]:
neg_class_prob_sorted = classifier.feature_log_prob_[0, :].argsort()
pos_class_prob_sorted = classifier.feature_log_prob_[1, :].argsort()

In [16]:
print('the top 5 features impact false binding: \n')
print(np.take(cv.get_feature_names(), neg_class_prob_sorted[:5]))

the top 5 features impact false binding: 

['lgggtg gggtge ggtgel gtgelf tgelff' 'ssgtsg sgtsgh gtsghn tsghne sghneq'
 'ggaggt gaggty aggtye ggtyeq gtyeqy' 'ggaggt gaggtd aggtdt ggtdtq gtdtqy'
 'ggaggr gaggrt aggrtd ggrtdt grtdtq']


In [17]:
print('the top 5 features impact true binding: \n')
print(np.take(cv.get_feature_names(), pos_class_prob_sorted[:5]))

the top 5 features impact true binding: 

['yywsgg ywsgga wsggae sggaee ggaeet' 'glentg lentge entgel ntgelf tgelff'
 'sslagp slagpr lagprg agprgg gprggy' 'sslagp slagpr lagpry agpryn gpryne'
 'glegre legret egretq gretqy retqyf']
